In [58]:
# packs
import os
import fnmatch
import numpy as np
import pandas as pd
from google.cloud import bigquery
import warnings

# ocultar warnings
warnings.filterwarnings('ignore')

#funciones
def consulta_archivo(patron, ruta='../querys'):
    """
    Ejecuta una consulta en BigQuery utilizando un archivo SQL que coincide con un patrón.

    Parameters:
    - patron (str): El patrón para buscar el archivo SQL.
    - ruta (str, optional): La ruta del directorio donde se buscarán los archivos SQL. 
                           Por defecto, es '../querys'.

    Returns:
    - pandas.DataFrame or None: El resultado de la consulta en forma de DataFrame de pandas. 
                               Si no se encuentra el archivo SQL, retorna None.

    Example:
    ```python
    patron_busqueda = 'megabrands'
    df_resultado = consulta_archivo(patron_busqueda)
    ```

    """
    # Encuentra el archivo SQL utilizando el patrón
    for nombre_archivo in os.listdir(ruta):
        if fnmatch.fnmatch(nombre_archivo, f'*{patron}*.sql'):
            # Construye la ruta completa al archivo SQL
            ruta_completa_sql = os.path.join(ruta, nombre_archivo)
            print(nombre_archivo)
            # Lee el contenido del archivo SQL
            with open(ruta_completa_sql, "r") as file:
                sql = file.read()

            # Ejecuta la consulta y convierte los resultados a un DataFrame de pandas
            cliente = bigquery.Client()
            df_resultado = cliente.query(sql).to_dataframe()
            print(df_resultado.head(2))
            print(df_resultado.shape)
            return df_resultado

    # Si no se encuentra ningún archivo que coincida con el patrón
    print('No se encontró el archivo SQL.')
    return None

def procesar_df_interacciones(df):
    """
    Función que procesa un DataFrame según las especificaciones dadas.

    Parámetros:
    - df: DataFrame a procesar.

    Returns:
    - DataFrame procesado.
    """
    print('Volumen inicial:')
    print(df.shape)
    print('Fase 1: Eliminando la variable "abi_brand"')
    df = df.drop('abi_brand', axis=1, errors='ignore')
    print('Fase 1 Done\n')

    print('Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"')
    limite_superior_nInteracciones = df['nInteracciones'].quantile(0.95)
    limite_superior_Dias_interaccion = df['Dias_interaccion'].quantile(0.95)
    
    df = df[(df['nInteracciones'] <= limite_superior_nInteracciones) & (df['Dias_interaccion'] <= limite_superior_Dias_interaccion)]
    
    print('Fase 2 Done\n')

    print('Fase 3: Seleccionando registros con "nInteracciones" > 1')
    df = df[df['nInteracciones'] > 1]
    print('Fase 3 Done\n')

    print('Fase 4: Aplicando la condición "brand lovers"')
    df = df[(df['nInteracciones'] > 4) | (df['Dias_interaccion'] < 75)]
    print('Fase 4 Done\n')

    print('Volumen final:')
    print(df.shape)
    df1 = df[['td_id1']] 
    df1.loc[:, 'origen'] = 'interaccion'
    return df1

def procesar_df_dtc(df):
    """
    Función que procesa un DataFrame según las especificaciones dadas.

    Parámetros:
    - df: DataFrame a procesar.

    Returns:
    - DataFrame procesado.
    """

    print('Volumen inicial:')
    print(df.shape)

    print('Fase 1: Filtrando valores superiores al percentil 99 en "ordenes" y "money"')
    limite_superior_ordenes = df['ordenes'].quantile(0.99)
    limite_superior_money = df['money'].quantile(0.99)
    
    df = df[(df['ordenes'] <= limite_superior_ordenes) & (df['money'] <= limite_superior_money)]
    
    print('Fase 1 done\n')

    print('Fase 2: Conservando registros superiores al percentil 1 en "money"')
    limite_inferior_money = df['money'].quantile(0.01)
    
    df = df[df['money'] >= limite_inferior_money]
    
    print('Fase 2 done\n')

    print('Fase 3: Seleccionando registros superiores o iguales al percentil 66 en "ordenes" y "money"')
    limite_superior_66 = df[['ordenes', 'money']].quantile(0.66)
    
    df = df[(df['ordenes'] >= limite_superior_66['ordenes']) & (df['money'] >= limite_superior_66['money'])]
    
    print('Fase 3 done\n')
    print('Volumen final:')
    print(df.shape)

    df1 = df[['td_id']] 
    df1.loc[:, 'origen'] = 'dtc'

    return df


In [73]:
# Llama a la función con el nombre de tu archivo SQL
df = consulta_archivo('nes_mega')
# Agrupar corona
df.loc[df['abi_brand'] == 'Corona Extra', 'abi_brand'] = 'Corona'

interacciones_mega.sql
                                              td_id1  nInteracciones  \
0  ffffac8e608e2c257507de669d5e84d62cccab4ce8653e...               2   
1  ffffa7a98bfb8d9dcfe437b51cc4b0f0bf36f806f55faf...               1   

   Dias_interaccion     abi_brand  
0               132        Corona  
1               104  Corona Extra  
(808127, 4)


In [75]:
df.td_id1.nunique()

808127

In [19]:
df.abi_brand.value_counts()

abi_brand
Corona           248196
Aguila           244624
Poker            199310
Club Colombia    116488
Name: count, dtype: int64

In [23]:

# Lista de marcas
marcas = ['Club Colombia', 'Poker', 'Corona', 'Aguila']

# Diccionario para almacenar DataFrames filtrados
dataframes_filtrados = {marca: df[df['abi_brand'].str.contains(marca)] for marca in marcas}

# Accede a los DataFrames individuales usando el nombre de la marca
club_df = dataframes_filtrados['Club Colombia']
poker_df = dataframes_filtrados['Poker']
corona_df = dataframes_filtrados['Corona']
aguila_df = dataframes_filtrados['Aguila']

In [46]:
club_lovers = procesar_df_interacciones(club_df)

Volumen inicial:
(116488, 4)
Fase 1: Eliminando la variable "abi_brand"
Fase 1 Done

Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"
Fase 2 Done

Fase 3: Seleccionando registros con "nInteracciones" > 1
Fase 3 Done

Fase 4: Aplicando la condición "brand lovers"
Fase 4 Done

Volumen final:
(19953, 3)


In [47]:
club_lovers.head()

,td_id1,origen
23,fffd89f1-da0a-414a-91c3-8f7d395c2c31,interaccion
47,fffb6838-c5d4-40e4-b9de-8af0635dff9f,interaccion
75,fff8cf9c7c603e23e5aadd03a5ed78aec24707f87ad873...,interaccion
88,fff7bfd701a95926596a72ae8e160a18cafb5015208463...,interaccion
197,ffeef0605759c9565dba5df3c5eade575b16ebd83ed0fd...,interaccion


In [48]:
club_lovers = procesar_df_interacciones(club_df)
poker_lovers = procesar_df_interacciones(poker_df)
corona_lovers = procesar_df_interacciones(corona_df)
aguila_lovers = procesar_df_interacciones(aguila_df)

Volumen inicial:
(116488, 4)
Fase 1: Eliminando la variable "abi_brand"
Fase 1 Done

Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"
Fase 2 Done

Fase 3: Seleccionando registros con "nInteracciones" > 1
Fase 3 Done

Fase 4: Aplicando la condición "brand lovers"
Fase 4 Done

Volumen final:
(19953, 3)
Volumen inicial:
(199310, 4)
Fase 1: Eliminando la variable "abi_brand"
Fase 1 Done

Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"
Fase 2 Done

Fase 3: Seleccionando registros con "nInteracciones" > 1
Fase 3 Done

Fase 4: Aplicando la condición "brand lovers"
Fase 4 Done

Volumen final:
(59667, 3)
Volumen inicial:
(248196, 4)
Fase 1: Eliminando la variable "abi_brand"
Fase 1 Done

Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"
Fase 2 Done

Fase 3: Seleccionando registros con "nInteracciones" > 1
Fase 3 Done

Fase 4: Aplicando la condición "brand lovers

In [66]:
# Cortes para tada
# Llama a la función con el nombre de tu archivo SQL
club = consulta_archivo('club_ta')
poker = consulta_archivo('poker_ta')
corona = consulta_archivo('corona_ta')
aguila = consulta_archivo('uila_tad')


club_tada.sql
                                               td_id  ordenes     money
0  00010ffddd535c3d04f67ad9f91b5493ab8a6d5bc41a36...        1  47520.00
1  0001a1de3f8c90902ae9b1dd22e5c78ae974887d05f3d1...        1  17792.86
(37510, 3)
poker_tada.sql
                                               td_id  ordenes      money
0  0001a1de3f8c90902ae9b1dd22e5c78ae974887d05f3d1...        1   15107.14
1               0001b68b-18d4-4a28-a7bd-d9ea6787c06d        4  161590.00
(68241, 3)
corona_tada.sql
                                               td_id  ordenes    money
0  00005beff72033e6d62d53c0be42b71a6474acfa502e40...        2  54600.0
1  0001dd3376517611a308369481b301d55a727b54e41cd5...        2  87400.0
(37399, 3)
aguila_tada.sql
                                               td_id  ordenes     money
0               000049dc-2f41-4ee9-b939-a45af13d06da        1   25200.0
1  00039052dee297eb16d2aea96a9286cd25d456c7e3beb4...        3  158320.0
(64841, 3)


In [77]:
pd.concat([club, poker, corona, aguila]).td_id.nunique()

150481

In [67]:
club_tada = procesar_df_dtc(club)
poker_tada = procesar_df_dtc(poker)
corona_tada = procesar_df_dtc(corona)
aguila_tada = procesar_df_dtc(aguila)

Volumen inicial:
(37510, 3)
Fase 1: Filtrando valores superiores al percentil 99 en "ordenes" y "money"
Fase 1 done

Fase 2: Conservando registros superiores al percentil 1 en "money"
Fase 2 done

Fase 3: Seleccionando registros superiores o iguales al percentil 66 en "ordenes" y "money"
Fase 3 done

Volumen final:
(9902, 3)
Volumen inicial:
(68241, 3)
Fase 1: Filtrando valores superiores al percentil 99 en "ordenes" y "money"
Fase 1 done

Fase 2: Conservando registros superiores al percentil 1 en "money"
Fase 2 done

Fase 3: Seleccionando registros superiores o iguales al percentil 66 en "ordenes" y "money"
Fase 3 done

Volumen final:
(17706, 3)
Volumen inicial:
(37399, 3)
Fase 1: Filtrando valores superiores al percentil 99 en "ordenes" y "money"
Fase 1 done

Fase 2: Conservando registros superiores al percentil 1 en "money"
Fase 2 done

Fase 3: Seleccionando registros superiores o iguales al percentil 66 en "ordenes" y "money"
Fase 3 done

Volumen final:
(9990, 3)
Volumen inicial:
(

In [72]:
# Concatenar los DataFrames a lo largo de la columna 'td_id'
df = pd.concat([aguila_lovers, aguila_tada], axis=0, ignore_index=True, sort=False)
df.shape

(126066, 5)

In [69]:
df.shape

(29855, 5)